# Tipología de Democracias en el Mundo (2025) – Análisis de Clustering

## Introducción
La democracia adopta múltiples formas y niveles de calidad.  
Este proyecto analiza **cómo se agrupan los países del mundo** según sus características democráticas, utilizando indicadores del prestigioso proyecto **V-Dem (Varieties of Democracy)**.

El objetivo es **descubrir tipologías de democracias** en el presente,
combinando técnicas de **Machine Learning no supervisado** con interpretación politológica.

**Dataset:** `V-Dem-CY-Core-v15.csv`  
- Versión 15 (marzo 2025)  
- Datos país–año con +1.000 indicadores de democracia, derechos y participación  
- Aquí trabajaremos sobre el **año más reciente disponible** para agrupar países por características similares

---

## Preguntas guía
- ¿Qué tipos de democracia se observan hoy en el mundo?  
- ¿Qué indicadores politológicos definen mejor esas diferencias?  
- ¿Qué países conforman democracias **liberales**, **electorales**, **delegativas** o **autoritarismos**?

---

## Objetivos
- Seleccionar indicadores centrales sobre calidad democrática (libertades, participación, instituciones).  
- Aplicar **PCA** para reducir dimensionalidad.  
- Entrenar **K-Means** (y/o DBSCAN) para identificar clusters.  
- Visualizar los agrupamientos en 2D y **analizar su interpretación politológica**.

---

## Alcance y supuestos
- Se utiliza información pública y consolidada de V-Dem.  
- Se trabaja a nivel **comparado** (todos los países con datos disponibles).  
- El resultado **no implica juicios normativos**, sino una clasificación basada en datos.

---

## Reproducibilidad
- Dependencias detalladas en `requirements.txt`  
- Estructura del proyecto basada en mejores prácticas de Data Science  
- Notebook y scripts alojados en `/notebooks` y `/src`

# 0. Configuración inicial

En esta sección se importan las librerías necesarias y se definen configuraciones visuales globales  
que se aplicarán a lo largo del notebook.

In [ ]:
# --- Importaciones principales ---
import pandas as pd
import numpy as np

# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Preprocesamiento ---
from sklearn.preprocessing import StandardScaler

# --- Reducción de dimensionalidad ---
from sklearn.decomposition import PCA

# --- Clustering ---
from sklearn.cluster import KMeans

# --- Evaluación del clustering ---
from sklearn.metrics import silhouette_score

print("Librerías importadas correctamente ✅")